In [ ]:
import time
from itertools import repeat
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import conceptnet_lite
from conceptnet_lite import Label, edges_between, edges_for

# ConceptNet Local Search

[Git Repo](https://github.com/ldtoolkit/conceptnet-lite)

In [ ]:
## CACHING (!!! execute this cell only once !!!)
#

node_cache_1 = {}
node_cache_2 = {}

#
##

In [ ]:
## Globals

src = 'input/fRAT.csv'
rat_frat = 'frat'
check = '2,3'

###

In [ ]:
def get_nodes_frat(node, depth=1):
    """ Given a node, get all the other nodes related to it
        The search is performed by looking at all the edges related to a particular node.

        Returns:
            dictionary of form:
                {"pick_someone's_brain": ['related_to'], 'blindly': ['related_to'], ...  'cross_purpose': ['related_to']}
    """
    if depth == 1:
        if node in node_cache_1.keys():
            return node_cache_1[node]
    elif depth == 2:
        if node in node_cache_2.keys():
            return node_cache_2[node]
    
    try:
        # edges starting from (our node)
        if node in node_cache_1.keys():
            nodes = node_cache_1[node]
        else:
            nodes = []
            for e in edges_for(Label.get(text=node, language='en').concepts, same_language=True):
                if e.start.text not in [node]:
                    nodes.append((e.start.text, e.relation.name))
                if e.end.text not in [node]:
                    nodes.append((e.end.text, e.relation.name))

        if depth == 2:
            nodes2 = []
            for n in nodes:
                for e in edges_for(Label.get(text=n[0], language='en').concepts, same_language=True):
                    if e.start.text not in [n[0]]:
                        nodes2.append((e.start.text, e.relation.name))
                    if e.end.text not in [n[0]]:
                        nodes2.append((e.end.text, e.relation.name))

        result = {}
        for tup in list(set(nodes if depth == 1 else nodes2)):
            if tup[0] not in result:
                result[tup[0]] = list()
                result[tup[0]].append(tup[1])
            else:
                result[tup[0]].append(tup[1])
        
        if depth == 1:
            node_cache_1[node] = result  
        else:
            node_cache_2[node] = result
        return result
    except Exception as e:
        msg = '!!!   No label for the node "{}"... Are you sure the spelling is correct?'.format(node)
        print(msg, e)
        return {}

def get_nodes_rat(word):
    """ Given a word, get all the compound words related to it as well as their relation name
        Compound words are basically being identified by the underscore (_)
    """
    # TODO: refactor the logic to be like frat
    try:
        result = []
        relation = []
        for e in edges_for(Label.get(text=word).concepts, same_language=True):
            if (e.start.text.find('_') != -1) & (e.start.text.find(word) != -1):
                result.append(e.start.text.replace(word, '').strip('_'))
                relation.append(e.relation.name)
            if (e.end.text.find('_') != -1) & (e.end.text.find(word) != -1):
                result.append(e.end.text.replace(word, '').strip('_'))
                relation.append(e.relation.name)

        joint_result = []
        for i in range(len(result)):
            if result[i].find('_') != -1:
                words = result[i].split('_')
                for word in words:
                    if word != '':
                        joint_result.append((word, relation[i]))
            else:
                joint_result.append((result[i], relation[i]))

        final_result = {}
        for tup in list(set(joint_result)):
            if tup[0] not in final_result:
                final_result[tup[0]] = list()
                final_result[tup[0]].append(tup[1])
            else:
                final_result[tup[0]].append(tup[1])
        return final_result
    except Exception as e:
        msg = '!!!   No label for the node "{}"... Are you sure the spelling is correct?'.format(node)
        print(msg, e)
        return {}

In [ ]:
def get_conceptnet():
    conceptnet_lite.connect('conceptnet_database.db')

def save_csv(rat_frat, check, output):
    output = pd.DataFrame(list(output))
    f_name = rat_frat + '_' + str(check) + '_conceptnet_search.xlsx'
    dIr = os.path.join('output', rat_frat)
    if not os.path.exists(dIr):
        os.mkdir(dIr)
    output.to_excel(os.path.join(dIr, f_name), index=False)
    print('Saved results in {}'.format(f_name))
    
def check_for(relation_dict, check_for, query):
    results = [set(relation_dict[key].keys()) for key in relation_dict.keys()]
    if '3' in check_for:
        yield results[0] & results[1] & results[2], [query[0], query[1], query[2]]
    if '2' in check_for:
        yield results[0] & results[1], [query[0], query[1]]
        yield results[0] & results[2], [query[0], query[2]]
        yield results[1] & results[2], [query[1], query[2]]

def get_output(solutions, query, relation_dict, ground_solution, has_solution):
    solutions = list(solutions)    
    relations = [] # both directions
    to_solution = [] # node -> solution
    from_solution = [] # solution -> node
    
    # build a relationship message for: (1) node, (2) relation (3) solution
    # For example:
    #
    # cues: antlers, doe, fawn
    # relation: related_to
    # solution: deer
    # relationship message: antler is related_to deer, doe is related_to to deer, fawn is related_to to deer
    for node in query:
        for sol in solutions:
            rel = ', '.join(relation_dict[node][sol.strip()]) # get the relationships for each node and solution
            relations.append(node + ' is "'+ rel + '" to ' + sol)

            for e in edges_between(Label.get(text=node, language='en').concepts, Label.get(text=sol, language='en').concepts):
                to_solution.append(e.start.text + ' is "' + e.relation.name + '" to ' + e.end.text)

    for solution in solutions:
        for node in query:
            for e in edges_between(Label.get(text=solution, language='en').concepts, Label.get(text=node, language='en').concepts):
                from_solution.append(e.start.text + ' is "' + e.relation.name + '" to ' + e.end.text)

    return {'FrAt': ', '.join(query),
            'ground solution': ground_solution,
            'solutions': ', '.join(solutions),
            'has_solution': has_solution,
            'relation': ' | '.join(relations),
            'relation_to_solution': ' | '.join(to_solution),
            'relation_from_solution': ' | '.join(from_solution)}

In [ ]:
def compute(items):
    index, query, df, args, output, accuracy = items
    get_nodes = get_nodes_rat if args[0] == 'rat' else get_nodes_frat
    solution = df.iloc[index].wans
    relation_dict = {}
    for node in query:
        relation_dict[node] = get_nodes(node)

    # the format of the relation_dict at this point would be
    #
    # { 'query_node': {'related_node_1': ['relation_1', 'relation_2'], ..., 'related_node_n': ['relation_1']}
    #  'question': {"pick_someone's_brain": ['related_to'], 'blindly': ['related_to'], ... 'cross_purpose': ['related_to']},
    #  'reply': {'repone': ['related_to'], ... 'sentences': ['related_to']},
    #  'solution': {'solutionism': ['derived_from', 'related_to'],... 'exhibit': ['related_to']}
    # }

    for result, quer in check_for(relation_dict, args[1], query):
        accuracy['total'] += 1
        has_solution = any(solution.lower().strip() in node for node in result)
        if has_solution:
            accuracy['tp'] += 1
            output.append(get_output(result, quer, relation_dict, solution, has_solution))
        else:
            print('Checking depth 2 for query {}...'.format(quer))
            relation_dict2 = {}
            for node in query:
                relation_dict2[node] = get_nodes(node, 2)
            for result2, quer2 in check_for(relation_dict2, args[1], query):
                has_solution = any(solution.lower().strip() in node for node in result2)
                if has_solution:
                    accuracy['tp'] += 1
                        
            output.append(get_output(result2, quer2, relation_dict2, solution, has_solution))
            

    return output, accuracy

In [ ]:
get_conceptnet()
output = []
accuracy = {
    'total': 0,
    'tp': 0
}
if rat_frat == 'rat': # csvs differ
    df = pd.read_csv(src)
else:
    df = pd.read_csv(src, sep=';')
queries = df.w1 + ' ' + df.w2 + ' ' + df.w3
queries = [list(map(lambda x: x.lower(), filter(len, line.split(' ')))) for line in queries]

# queries format
# [['question', 'reply', 'solution'], ... ['fault', 'incorrect', 'unjust']]
for item in tqdm(zip(range(0, len(queries)),
                queries,
                repeat(df),
                repeat((rat_frat, check)),
                repeat(output),
                repeat(accuracy))):
    output, accuracy = compute(item)

output.append({'Accuracy': str(round(100*accuracy['tp']/accuracy['total'], 2)) + '%'})
save_csv(rat_frat, check, output)